In [ ]:
import pandas as pd 
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import geopandas as gpd
import datetime as dt
import copy
import matplotlib.pyplot as plt

In [ ]:
def add_noise(cell):
    """
    This function add noise to the input variable
    
    input: a number--fload or integer
    output: a noisy number
    """
    cell = cell + 0.005*np.random.randn()
    return cell

In [ ]:
def dbscan_(epsilon_m, min_samples, current_id):
    eps = epsilon_m * 0.000009
    data = current_id.copy()
    
    clustering = DBSCAN(
        eps = eps, min_samples = min_samples).fit(data[['Long','Lat']])
    
    data.loc[:, 'clusterIndex'] = clustering.labels_
    return data

In [ ]:
def agg_cluster(epsilon_m, rec_points):
    eps = epsilon_m * 0.000009
    data = rec_points.copy()
    
    add_clsut = AgglomerativeClustering(n_clusters=None, 
                                            linkage='single', 
                                                distance_threshold=eps).fit(data[['Long','Lat']])
    
    data.loc[:, 'rec_cluster_index'] = add_clsut.labels_
    return data

In [ ]:
def dbscan2_(epsilon_m2, min_samples2, rec_points):
    eps = epsilon_m2 * 0.000009
    data = rec_points.copy()
    
    clustering2 = DBSCAN(
        eps = eps, min_samples = min_samples2).fit(data[['Long','Lat']])
    
    data.loc[:, 'clusterIndex2'] = clustering2.labels_
    return data

In [ ]:
# if data frame became large u have to change block size
raw_data = pd.read_csv(
    r'C:\Users\Rahmani\Desktop\MobileData\Data\Data20000fix.csv',
    parse_dates=[0])


stay_points = pd.read_csv(r'C:\Users\Rahmani\Desktop\MobileData\Data\3000_staypoint.csv', 
                          parse_dates=[0,1])

In [ ]:
tehran_shp = gpd.read_file(r'C:\Users\Rahmani\Desktop\MobileData\TehranGeorefed\Tehran\tehran.shp')

In [ ]:
# Reading Tehran Shapefile
tehran_manategh = gpd.read_file(r'C:\Users\Rahmani\Desktop\MobileData\Tehran_SHP_New\Manategh_22\manategh.shp',crs={'init': 'epsg:4326'})
tehran_manategh.to_crs(epsg=4326, inplace=True)
tehran_manategh.drop(tehran_manategh.columns[[-2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13]], axis=1, inplace = True)
tehran_manategh

In [ ]:
# Reading Tehran Shapefile
tehran_navahi = gpd.read_file(r'C:\Users\Rahmani\Desktop\MobileData\Tehran_SHP_New\Navahi_124\navahi.shp')
tehran_navahi.to_crs(epsg=4326, inplace=True)
tehran_navahi.drop(tehran_navahi.columns[[-4, -5, -6, -7]], axis=1, inplace = True)
tehran_navahi

In [ ]:
# Adding noise
raw_data['noisy_lat'] = raw_data['Lat'].apply(add_noise)
raw_data['noisy_long'] = raw_data['Long'].apply(add_noise)

In [ ]:
raw_data.loc[:, 'hour'] = raw_data.loc[:,'Date_Time'].dt.hour
raw_data.loc[:, 'weekday'] = raw_data.loc[:,'Date_Time'].dt.day_name()

In [ ]:
raw_data.sort_values(by=['ID', 'Date_Time'], inplace=True)
raw_data.reset_index(inplace = True, drop = True)

# Some Special IDs
Karaj:
ID: 19

weird clusters:
ID: 21

In [ ]:
raw_data.head()

In [ ]:
stay_points.head()

In [ ]:
raw_data_groupby = raw_data.groupby(by=['ID'])
counter = 0
clustered_data_by_id = {}
homeless_people = {}
workless_people = {}
recreation_points = pd.DataFrame()
home_locations_by_id = {}
work_locations_by_id = {}

for ID in raw_data.ID.unique():
    counter += 1
    current_id = raw_data_groupby.get_group(ID)
    current_id_clustered = dbscan_(1200, 7, current_id)
    current_id_clustered.loc[:, 'clust_label'] = 'Not Known'
    #home = current_id_clustered.loc[current_id_clustered['weekday'] != 'Friday'][['clusterIndex']].between_time('23:30', '06:00').mode()
    #current_id_clusters_pivot = current_id_clustered.groupby(['clusterIndex'])[['ID']].count().sort_values('ID', ascending=False)
    # ===============================================
    # Finding home cluster
    index = pd.DatetimeIndex(current_id_clustered['Date_Time'].loc[(current_id_clustered['weekday'] != 'Friday')])
    home_clust = current_id_clustered.loc[(
                    current_id_clustered['weekday'] != 'Friday')][['clusterIndex']].iloc[
                        index.indexer_between_time('00:00','06:00')].mode()
    # Labeling home cluster
    if not home_clust.empty:
        current_id_clustered.loc[current_id_clustered['clusterIndex'] == home_clust.iat[0,0], 'clust_label'] = 'Home'
    else:
        homeless_people['{}'.format(ID)] = current_id_clustered
    
    # ===============================================
    # Finding work cluster
    index = pd.DatetimeIndex(current_id_clustered['Date_Time'].loc[
        (current_id_clustered['weekday'] != 'Friday') 
            & ((current_id_clustered['clust_label'] != 'Home'))])
    
    work_clust = current_id_clustered.loc[(
                    current_id_clustered['weekday'] != 'Friday') 
                        & (current_id_clustered['clust_label'] != 'Home')][['clusterIndex']].iloc[
                            index.indexer_between_time('09:00','17:00')].mode()
    # Labeling work cluster
    if not work_clust.empty:
        if (current_id_clustered.clusterIndex.values == work_clust.iat[0,0]).sum() > 8:
            current_id_clustered.loc[current_id_clustered['clusterIndex'] == work_clust.iat[0,0], 'clust_label'] = 'Work'
    else:
        workless_people['{}'.format(ID)] = current_id_clustered
    # ===============================================
    # Calculating average location for home and work
    current_id_hw_loc = current_id_clustered.groupby('clust_label')[['Lat', 'Long']].mean()
    current_id_hw_loc.reset_index(inplace = True)
    current_id_home_loc = current_id_hw_loc.loc[current_id_hw_loc['clust_label'] == 'Home']
    current_id_work_loc = current_id_hw_loc.loc[current_id_hw_loc['clust_label'] == 'Work']
    home_locations_by_id['{}'.format(ID)] = current_id_home_loc
    work_locations_by_id['{}'.format(ID)] = current_id_work_loc

    # ===============================================
    clustered_data_by_id['{}'.format(ID)] = current_id_clustered
    
    # Recreational Filter 
    index = pd.DatetimeIndex(current_id_clustered['Date_Time'].loc[
        ((current_id_clustered['weekday'] == 'Friday') | (current_id_clustered['weekday'] == 'Thursday'))
            & (current_id_clustered['clust_label'] != 'Home') 
                & (current_id_clustered['clust_label'] != 'Work')])
    
    current_id_recreation_points = current_id_clustered.loc[
                    ((current_id_clustered['weekday'] == 'Friday') | (current_id_clustered['weekday'] == 'Thursday'))
                        & (current_id_clustered['clust_label'] != 'Home') 
                            & (current_id_clustered['clust_label'] != 'Work')].iloc[
                                index.indexer_between_time('19:00','23:59')]
    if not current_id_recreation_points.empty:
        if not home_clust.empty:
            current_id_recreation_points = current_id_recreation_points.assign(home_lat= current_id_home_loc.at[0, 'Lat'], 
                                                                               home_long= current_id_home_loc.at[0, 'Long'])
            recreation_points = recreation_points.append(current_id_recreation_points, ignore_index=True)

#     if counter == 2:
#         break;
        

# for key in clustered_data_by_Id:
#     clusters_sorted = copy.copy(clustered_data_by_Id[key])
    

In [ ]:
recreation_points.to_csv('all_recreation_points.csv')

# Clustering Recreational Points to indentify hotspot areas

In [ ]:
# recreation_points_clustered = dbscan_(500, 100, recreation_points) # DBCAN does not work well here.
recreation_points_clustered = agg_cluster(500, recreation_points)

In [ ]:
# Creating the GeoPandas dartaframe (shapefile)
recreation_points_clustered_shp = gpd.GeoDataFrame(recreation_points_clustered, 
                                                   geometry = gpd.points_from_xy(recreation_points_clustered.Long, 
                                                                                 recreation_points_clustered.Lat), 
                                                                                   crs={'init': 'epsg:4326'})

In [ ]:
# Sort clusters based on their number of members
recreation_points_clustered_pivot = recreation_points_clustered_shp.groupby(['rec_cluster_index'])[['ID']].count().sort_values('ID', ascending=False)

# recreation_points_clustered.groupby(['clusterIndex2']).agg(['count']) # if you use DBSCAN here, you  should change to clusters index columns
# in order to prevent overwriting new cluster indeces on previous indeces from home-work clustering session.

In [ ]:
# Filtering top_n clusters

top_rec_points = recreation_points_clustered_pivot.head(10) # You can change 10 to X to find the top X areas
top_rec_points.reset_index(inplace=True) # index to column 
top_10_areas = recreation_points_clustered_shp.loc[(recreation_points_clustered_shp.rec_cluster_index.isin(top_rec_points['rec_cluster_index']))]

In [ ]:
top_10_areas

In [ ]:
# Ploting top_n areas on the map
base = tehran_manategh.plot(color='white', edgecolor='black')
top_10_areas.plot(ax=base, marker='o', column='rec_cluster_index', markersize=5, categorical=True, legend = True);

In [ ]:
# Saving top_n areas into shp and gpkg
top_10_areas.to_file("recreation_points_10top_20000.gpkg", layer='rec_points', driver="GPKG")

In [ ]:
# Saving to CSV
top_10_areas.to_csv("recreation_points_10top_20000.csv", mode='w', columns=['Date_Time','ID','Lat','Long','hour','weekday','home_lat','home_long', 'rec_cluster_index'], index=False)

# Recreational Points Spatial Join

In [ ]:
top_10_areas = pd.read_csv("recreation_points_10top_20000.csv", parse_dates=[0])

In [ ]:
top_10_areas

In [ ]:
# Replacing the location of each recreation cluster members with the centroid of the cluster
# By centroid we mean the average location of all cluter members

top_10_areas['Long'] = top_10_areas.groupby(['rec_cluster_index'])['Long'].transform('mean') 
top_10_areas['Lat'] = top_10_areas.groupby(['rec_cluster_index'])['Lat'].transform('mean') 

In [ ]:
top_10_areas_lat_longs = top_10_areas.groupby('rec_cluster_index')[['Lat', 'Long']].mean()

In [ ]:
top_10_areas_lat_longs

### Manategh 22

In [ ]:
top_10_areas_shp = gpd.GeoDataFrame(top_10_areas, 
                                        geometry = gpd.points_from_xy(top_10_areas.Long, 
                                                                        top_10_areas.Lat), 
                                                                            crs={'init': 'epsg:4326'})
top_10_areas_shp.set_crs(epsg=4326, inplace=True, allow_override=True)
top_10_areas_in_manategh = gpd.sjoin(top_10_areas_shp, tehran_manategh, how="left", op='within')
top_10_areas_in_manategh.rename(columns={"IDMAN": "recreation_mantaghe"}, inplace = 'True')
top_10_areas_in_manategh.drop(columns=['index_right'], inplace = True) 

top_10_areas_in_manategh = gpd.GeoDataFrame(top_10_areas_in_mahalat, 
                                        geometry = gpd.points_from_xy(top_10_areas_in_manategh.home_long, 
                                                                        top_10_areas_in_manategh.home_lat), 
                                                                            crs={'init': 'epsg:4326'})
top_10_areas_in_manategh.set_crs(epsg=4326, inplace=True, allow_override=True)

top_10_areas_in_manategh = gpd.sjoin(top_10_areas_in_mahalat, tehran_manategh, how="left", op='within')
top_10_areas_in_manategh.rename(columns={"IDMAN": "home_mantaghe"}, inplace = 'True')
top_10_areas_in_manategh.drop(columns=['index_right'], inplace = True) 

### Navahi 124

In [ ]:
top_10_areas_shp = gpd.GeoDataFrame(top_10_areas, 
                                        geometry = gpd.points_from_xy(top_10_areas.Long, 
                                                                        top_10_areas.Lat), 
                                                                            crs={'init': 'epsg:4326'})
top_10_areas_shp.set_crs(epsg=4326, inplace=True, allow_override=True)
top_10_areas_in_navahi = gpd.sjoin(top_10_areas_shp, tehran_navahi, how="left", op='within')
top_10_areas_in_navahi.rename(columns={"NAVAHI": "recreation_mantaghe"}, inplace = 'True')
top_10_areas_in_navahi.drop(columns=['index_right'], inplace = True) 

top_10_areas_in_navahi = gpd.GeoDataFrame(top_10_areas_in_navahi, 
                                        geometry = gpd.points_from_xy(top_10_areas_in_navahi.home_long, 
                                                                        top_10_areas_in_navahi.home_lat), 
                                                                            crs={'init': 'epsg:4326'})
top_10_areas_in_navahi.set_crs(epsg=4326, inplace=True, allow_override=True)

top_10_areas_in_navahi = gpd.sjoin(top_10_areas_in_navahi, tehran_navahi, how="left", op='within')
top_10_areas_in_navahi.rename(columns={"NAVAHI": "home_mantaghe"}, inplace = 'True')
top_10_areas_in_navahi.drop(columns=['index_right'], inplace = True) 

#### pivot table

In [ ]:
# table = pd.pivot_table(top_10_areas_in_navahi, values='NAVAHI_right', index=['NAVAHI_left'],
#                     columns=['NAVAHI_right'], aggfunc=np.sum)
recreation_OD = top_10_areas_in_navahi.pivot_table(index='home_mantaghe',
                    columns='recreation_mantaghe', values = 'ID', aggfunc=len, fill_value=0)

In [ ]:
vector = pd.DataFrame(recreation_OD.stack())
vector

In [ ]:
a = pd.merge(vector, tehran_navahi[['NAVAHI','Lat','Long']], left_on = ['home_mantaghe'],
                   right_on = ['NAVAHI'], 
                   how = 'left')
b = pd.merge(vector, tehran_navahi[['NAVAHI','Lat','Long']], left_on = ['recreation_mantaghe'],
                   right_on = ['NAVAHI'], 
                   how = 'left')

In [ ]:
a


In [ ]:
b

In [ ]:
c = pd.concat([a, b], axis=1, sort=False)
c

In [ ]:
vector.to_csv('vectorized.csv')
recreation_OD.to_csv('od.csv')
c.to_csv('keplger_input_test.csv')

# Ploting Stay points for Current ID

In [ ]:
# Creating Person shapefile
current_id_points = gpd.GeoDataFrame(
        current_id_clustered, geometry = gpd.points_from_xy(
                                            current_id_clustered.Long, 
                                                current_id_clustered.Lat),  
                                                    crs={'init': 'epsg:4326'})

base = tehran_manategh.plot(color='white', edgecolor='black')
current_id_points.plot(ax=base, marker='o', column='clusterIndex', markersize=5, categorical=True, legend = True);

In [ ]:
# Creating Noisy Points Shapefile
# current_Id_clustered['noisy_lat'] = current_Id_clustered['Lat'].apply(add_noise)
# current_Id_clustered['noisy_long'] = current_Id_clustered['Long'].apply(add_noise)

current_id_clustered_points = gpd.GeoDataFrame(
    current_id_clustered, geometry = gpd.points_from_xy(
        current_id_clustered.noisy_long, current_id_clustered.noisy_lat),  crs={'init': 'epsg:4326'})

base = tehran_manategh.plot(color='white', edgecolor='black')
current_id_clustered_points.plot(ax=base, marker='o', column='clusterIndex', markersize=5, categorical=True, legend = True);

# Ploting the cluster time histogram 

In [ ]:
# Defining the weekdays in order to seperate workdays and weekdays
current_id_clustered['weekday'] = current_id_clustered[['Date_Time']].apply(lambda x: dt.datetime.strftime(x['Date_Time'], '%A'), axis=1)

# Defining the cluster number for which we want to plot the histogram
desired_cluster = copy.copy(current_id_clustered[current_id_clustered['clusterIndex'] == 2])

# Filtering Workdays
desired_cluster_workdays = copy.copy(desired_cluster[(desired_cluster['weekday'] != 'Friday') & (desired_cluster['weekday'] != 'Thursday')])
# Filtering Weekends
desired_cluster_weekends = copy.copy(desired_cluster[(desired_cluster['weekday'] == 'Friday')])
# all days
desired_cluster_all_days = copy.copy(desired_cluster)

In [ ]:
# Defining the hour of connection
desired_cluster_all_days.loc[:, 'hour'] = desired_cluster_all_days.loc[:,'Date_Time'].dt.hour

# Defining the hour of connection
desired_cluster_workdays.loc[:, 'hour'] = desired_cluster_workdays.loc[:,'Date_Time'].dt.hour

# Defining the hour of connection
desired_cluster_weekends.loc[:, 'hour'] = desired_cluster_weekends.loc[:,'Date_Time'].dt.hour

# Ploting the histogram
desired_cluster_all_days.hist(column='hour', bins=23, grid=False, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)
plt.title('All Days Histogram')

# Ploting the histogram
desired_cluster_workdays.hist(column='hour', bins=23, grid=False, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)
plt.title('Work Days Histogram')

desired_cluster_weekends.hist(column='hour', bins=23, grid=False, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)
plt.title('Weekends Histogram')
# Method 2: plt.hist(desired_cluster['hour'], bins = 24)

In [ ]:
desired_cluster_workdays.groupby(['hour']).agg(['count'])

In [ ]:
# Creating Person shapefile
current_ID_Points = gpd.GeoDataFrame(
        current_Id_clustered, geometry = gpd.points_from_xy(
                                            current_Id_clustered.Long, 
                                                current_Id_clustered.Lat),  
                                                    crs={'init': 'epsg:4326'})

base = tehran_manategh.plot(color='white', edgecolor='black')
current_ID_Points.plot(ax=base, marker='o', column='clusterIndex', markersize=5, categorical=True, legend = True);

In [ ]:
# Creating Noisy Points Shapefile
current_Id_clustered['noisy_lat'] = current_Id_clustered['Lat'].apply(add_noise)
current_Id_clustered['noisy_long'] = current_Id_clustered['Long'].apply(add_noise)

current_Id_clustered_Points = gpd.GeoDataFrame(
    current_Id_clustered, geometry = gpd.points_from_xy(
        current_Id_clustered.noisy_long, current_Id_clustered.noisy_lat),  crs={'init': 'epsg:4326'})

base = tehran_manategh.plot(color='white', edgecolor='black')
current_Id_clustered_Points.plot(ax=base, marker='o', column='clusterIndex', markersize=5, categorical=True, legend = True);